In [1]:
!pip install -q keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import os

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
X_train = x_train.reshape(-1,28, 28, 1)
X_test = x_test.reshape(-1,28,28,1)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print(X_train.shape)

(48000, 28, 28, 1)


In [6]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        batch_size = trial.hyperparameters.Choice('batch_size', [16, 32])

        history = model.fit(
            X, y,
            batch_size=batch_size,
            validation_data=validation_data,
            **fit_kwargs
        )

        # train accuracy 기준으로 튜닝
        train_acc = history.history.get("accuracy")
        if train_acc:
            self.oracle.update_trial(trial.trial_id, {"accuracy": train_acc[-1]})
        else:
            raise ValueError("accuracy가 history에 없습니다.")

        self.save_model(trial.trial_id, model)

    # 🔥 save_model 메서드 구현
    def save_model(self, trial_id, model, **kwargs):
        model.save(f"model_{trial_id}.h5")

In [7]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

마지막으로 keras_tuner를 정의하고 탐색하는것까지 보도록 하겠습니다.

저는 이번 모델의 경우 BayesianOptimizationOracle을 사용할 예정이며 목표는 accuracy와 max로 둘 예정입니다. 실제 trial은 10번으로 지정할 것입니다.

hypermodel은 build_model을 넣어주시고 project이름도 작성해주세요.

마지막으로 search함수에 X_train, Y_train, validation data, epoch을 넣고 탐색합니다!

In [8]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='my_keras_tuner')

# 해당 모델 학습시간은 약 10분정도 걸립니다!
my_keras_tuner.search(
    X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 01m 00s]
accuracy: 0.9757083058357239

Best accuracy So Far: 0.9886875152587891
Total elapsed time: 00h 11m 53s


In [9]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_65 (Conv2D)              │ (None, 26, 26, 102)    │         1,020 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_66 (Conv2D)              │ (None, 24, 24, 42)     │        38,598 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_67 (Conv2D)              │ (None, 22, 22, 67)     │        25,393 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_68 (Conv2D)              │ (None, 20, 20, 37)     │        22,348 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_69 (Conv2D)              │ (None, 18, 18, 52)     │        17,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 16848)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │     4,313,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,420,641 (16.86 MB)

 Trainable params: 4,420,641 (16.86 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.fit(X_train, y_train, batch_size=32, epochs = 5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.8571 - loss: 0.7532
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9817 - loss: 0.0596
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9884 - loss: 0.0358
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.9910 - loss: 0.0288
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9944 - loss: 0.0175


In [11]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9789 - loss: 0.0940


[0.07223457843065262, 0.9830999970436096]

In [13]:
save_path = './model'
os.makedirs(save_path, exist_ok=True)
model.save(os.path.join(save_path, 'best_model.keras'))

In [15]:
model = tf.keras.models.load_model('./model/best_model.keras')
model.export('./serving_model/1')  # ✅ Keras 3 방식: export()로 저장

Saved artifact at './serving_model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='inputs')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  139320276268048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655547024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655550480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655551056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655550288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655551632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655550864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655551824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655551440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317655544528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139317468181712: Ten

In [16]:
import shutil
shutil.make_archive('/content/serving_model', 'zip', '/content/serving_model')

from google.colab import files
files.download('/content/serving_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>